In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import openpyxl
import xlrd
from rdkit import Chem
from rdkit.Chem import inchi
import papyrus_scripts
from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardizer as Papyrus_standardizer
from papyrus_structure_pipeline import standardize

[<class 'str'>, typing.Iterable[str]]
[typing.Tuple[int, int], typing.Tuple[int]]
[<class 'pyparsing.results.ParseResults'>, typing.Sequence[pyparsing.results.ParseResults]]
[typing.Callable[[], typing.Any], typing.Callable[[pyparsing.results.ParseResults], typing.Any], typing.Callable[[int, pyparsing.results.ParseResults], typing.Any], typing.Callable[[str, int, pyparsing.results.ParseResults], typing.Any]]
[typing.Callable[[], bool], typing.Callable[[pyparsing.results.ParseResults], bool], typing.Callable[[int, pyparsing.results.ParseResults], bool], typing.Callable[[str, int, pyparsing.results.ParseResults], bool]]
[typing.Callable[[str, int, ForwardRef('ParserElement'), bool], NoneType], <class 'NoneType'>]
[typing.Callable[[str, int, int, ForwardRef('ParserElement'), pyparsing.results.ParseResults, bool], NoneType], <class 'NoneType'>]
[typing.Callable[[str, int, ForwardRef('ParserElement'), Exception, bool], NoneType], <class 'NoneType'>]
[<class 'pyparsing.results.ParseResults'>

[13:26:37] Initializing Normalizer


In [3]:

from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

Query data from Papyrus

In [3]:

from papyrus_scripts import download_papyrus
download_papyrus(only_pp=False)

Latest version: 05.6
Number of files to be downloaded: 10
Total size: 8.18GB


In [2]:
from papyrus_scripts import read_papyrus
data = read_papyrus(plusplus=False,chunksize=1_000_000)

[<class 'bytes'>, typing.List[bytes]]
[typing.List[charset_normalizer.models.CharsetMatch], <class 'NoneType'>]
[ForwardRef('CharsetMatch'), <class 'NoneType'>]
[<class 'bytes'>, <class 'bytearray'>]
[typing.List[str], <class 'NoneType'>]
[<class 'str'>, <class 'bytes'>, <class 'os.PathLike'>]
[<class 'os.PathLike'>, <class 'str'>, <class 'typing.BinaryIO'>, <class 'bytes'>]
[<class 'str'>, <class 'float'>]
[<class 'str'>, <class 'float'>, <class 'NoneType'>]
[<class 'bytes'>, <class 'str'>]
[<class 'str'>, <class 'bytes'>, <class 'bytearray'>]
[typing.Mapping[str, str], <class 'NoneType'>]
[<class 'str'>, <class 'pathlib.Path'>]
[typing.Mapping[str, typing.Any], <class 'NoneType'>]
[<class 'str'>, typing.Sequence[str]]
[ForwardRef('botocore.client.BaseClient'), <class 'NoneType'>]
[~X, <class 'NoneType'>]
[typing.Type[~X], <class 'NoneType'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef(

2024-02-14 13:14:00,165	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[<class 'str'>, <class 'logging.Logger'>]
[ForwardRef('RepeatedScalarFieldContainer[_T]'), typing.Iterable[~_T]]
[<class 'int'>, <class 'NoneType'>]
[<class 'int'>, <class 'slice'>]
[ForwardRef('RepeatedCompositeFieldContainer[_T]'), typing.Iterable[~_T]]
[~_V, <class 'NoneType'>]
[~_V, ~_T]
[<class 'dict'>, <class 'NoneType'>]
[<class 'bool'>, <class 'NoneType'>]
[typing.IO[~AnyStr], <class 'NoneType'>]
[typing.List[int], <class 'NoneType'>]
[typing.Callable[[], NoneType], <class 'NoneType'>]
[ForwardRef('_BarManager'), <class 'NoneType'>]
[typing.Iterable, <class 'NoneType'>]
[typing.List[str], <class 'NoneType'>]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv')]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv'), <class 'NoneType'>]
[ForwardRef('grpc.ChannelCredentials'), <class 'NoneType'>]
[typing.Dict[str, typing.Any], <class 'NoneType'>]
[<class 'float'>, <class 'NoneType'>]
[typing.Sequence[typing.Tuple[str, typing.Any]], <class 'NoneType'>]
[<class 'str'>, <class '

In [3]:
efflux_ac_list = ['P08183','Q9UNQ0','P33527','Q92887','015438','015439','015440','095255']

In [4]:
from papyrus_scripts import read_protein_set
protein_data = read_protein_set()

In [5]:
from papyrus_scripts import keep_quality
from papyrus_scripts import keep_accession

filter1=keep_quality(data,'High')
filter2=keep_accession(filter1, efflux_ac_list)

In [6]:
from papyrus_scripts import consume_chunks
efflux = consume_chunks(filter2)

0it [00:00, ?it/s]

In [7]:
print(len(efflux))
efflux.head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,type_other,Activity_class,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD
381,AACDPQIFRBEZFP_on_Q9UNQ0_WT,High,ExCAPE-DB;ChEMBL31;ChEMBL31,10612087;ChEMBL31.compound.172357;ChEMBL31.com...,COc1cccc(C=CC(=O)c2ccc(O)cc2O)c1,AACDPQIFRBEZFP,AACDPQIFRBEZFP-UHFFFAOYSA-N,InChI=1S/C16H14O4/c1-20-13-4-2-3-11(9-13)5-8-1...,"""AuxInfo=1/0/N:1,5,6,4,8,14,13,9,20,17,7,15,3,...",Q9UNQ0_WT,...,1;0;0,NaN,=,5.712; 5.710; 5.550,5.657,0.076,0.054,3.0,5.710,0.002
2568,AAOKPBUEBIBSRX_on_P33527_WT,High,ChEMBL31,ChEMBL31.compound.301291,N#Cc1c(C2CCCCCC2)n(C2CCCCCC2)c2c1ncnc2N1CCN(C(...,AAOKPBUEBIBSRX,AAOKPBUEBIBSRX-UHFFFAOYSA-N,InChI=1S/C38H46N6/c39-27-33-34-37(44(32-21-13-...,"""AuxInfo=1/0/N:8,9,16,17,34,40,7,10,33,35,39,4...",P33527_WT,...,0,NaN,=,6.000; 5.710,5.855,0.145,0.145,2.0,5.855,0.145
2569,AAOKPBUEBIBSRX_on_Q9UNQ0_WT,High,ChEMBL31,ChEMBL31.compound.301291,N#Cc1c(C2CCCCCC2)n(C2CCCCCC2)c2c1ncnc2N1CCN(C(...,AAOKPBUEBIBSRX,AAOKPBUEBIBSRX-UHFFFAOYSA-N,InChI=1S/C38H46N6/c39-27-33-34-37(44(32-21-13-...,"""AuxInfo=1/0/N:8,9,16,17,34,40,7,10,33,35,39,4...",Q9UNQ0_WT,...,0,NaN,=,5.060,5.060,0.000,0.000,1.0,5.060,0.000
5636,ABFNXIZZKNVMDN_on_P08183_WT,High,ChEMBL31,ChEMBL31.compound.301870,COc1cccc(C=CC(=O)c2ccc(Nc3nc(-c4cc(OC)c(OC)cc4...,ABFNXIZZKNVMDN,ABFNXIZZKNVMDN-UHFFFAOYSA-N,InChI=1S/C32H27N3O4/c1-37-25-8-6-7-21(19-25)11...,"""AuxInfo=1/1/N:1,27,24,34,33,5,6,4,35,32,8,13,...",P08183_WT,...,0,NaN,=,4.840,4.840,0.000,0.000,1.0,4.840,0.000
5637,ABFNXIZZKNVMDN_on_Q9UNQ0_WT,High,ChEMBL31,ChEMBL31.compound.301870,COc1cccc(C=CC(=O)c2ccc(Nc3nc(-c4cc(OC)c(OC)cc4...,ABFNXIZZKNVMDN,ABFNXIZZKNVMDN-UHFFFAOYSA-N,InChI=1S/C32H27N3O4/c1-37-25-8-6-7-21(19-25)11...,"""AuxInfo=1/1/N:1,27,24,34,33,5,6,4,35,32,8,13,...",Q9UNQ0_WT,...,0,NaN,=,6.440,6.440,0.000,0.000,1.0,6.440,0.000


#Check for molecules that are present both in training and validation dataset: remove from validation

In [11]:
train = pd.read_csv('train_data/kadar_efflux_train.csv', delimiter=',', index_col=0)
mask = efflux['connectivity'].isin(train['inchi_connectivity'])
common_values_list = efflux[mask]['connectivity'].tolist()
print(f'Common: {len(common_values_list)}')

efflux_val = efflux[~mask]
print(f'Final validation: {len(efflux_val)}')

In [15]:
#Keep the item with the highest pchembl value
val_efflux = efflux_val.loc[efflux_val.groupby('InChIKey')['pchembl_value_Mean'].idxmax()]

In [16]:
#Classify

def compare_values(row):
    if row['pchembl_value_Mean'] <= 5:
        return 'Non-substrate'
    elif row['pchembl_value_Mean'] > 5:
        return 'Substrate'
    else:
        return '??'

val_efflux['status_efflux'] = val_efflux.apply(compare_values, axis=1)


,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Activity_class,relation,pchembl_value,pchembl_value_Mean,pchembl_value_StdDev,pchembl_value_SEM,pchembl_value_N,pchembl_value_Median,pchembl_value_MAD,status_efflux
5637,ABFNXIZZKNVMDN_on_Q9UNQ0_WT,High,ChEMBL31,ChEMBL31.compound.301870,COc1cccc(C=CC(=O)c2ccc(Nc3nc(-c4cc(OC)c(OC)cc4...,ABFNXIZZKNVMDN,ABFNXIZZKNVMDN-UHFFFAOYSA-N,InChI=1S/C32H27N3O4/c1-37-25-8-6-7-21(19-25)11...,"""AuxInfo=1/1/N:1,27,24,34,33,5,6,4,35,32,8,13,...",Q9UNQ0_WT,...,NaN,=,6.440,6.440,0.000,0.000,1.0,6.440,0.000,Substrate
43528,AJAJIMATEVMQTD_on_Q9UNQ0_WT,High,ChEMBL31,ChEMBL31.compound.225658,COc1c(OC)cc(-c2nc3c(cccc3)c(Nc3cccc(O)c3)n2)cc1,AJAJIMATEVMQTD,AJAJIMATEVMQTD-UHFFFAOYSA-N,InChI=1S/C22H19N3O3/c1-27-19-11-10-14(12-20(19...,"""AuxInfo=1/1/N:1,6,14,15,21,20,22,13,16,27,28,...",Q9UNQ0_WT,...,NaN,=,6.850; 6.960,6.905,0.055,0.055,2.0,6.905,0.055,Substrate
44271,AJFDIQBBWVOEGI_on_Q92887_WT,High,ChEMBL31,ChEMBL31.compound.308888,CCCCn1c(C=C2Oc3c(C2=O)c(O)cc(O)c3)c(C)c2c1cccc2,AJFDIQBBWVOEGI,AJFDIQBBWVOEGI-UHFFFAOYSA-N,InChI=1S/C22H21NO4/c1-3-4-9-23-16-8-6-5-7-15(1...,"""AuxInfo=1/0/N:1,21,2,3,26,25,27,24,4,16,19,7,...",Q92887_WT,...,NaN,=,5.090,5.090,0.000,0.000,1.0,5.090,0.000,Substrate
44525,AJGKQXIVAXXUTP_on_Q92887_WT,High,ChEMBL31,ChEMBL31.compound.308880,CCCCn1c(C)c(C=C2Oc3c(C2=O)c(O)cc(O)c3)c2c1cccc2,AJGKQXIVAXXUTP,AJGKQXIVAXXUTP-UHFFFAOYSA-N,InChI=1S/C22H21NO4/c1-3-4-9-23-13(2)16(15-7-5-...,"""AuxInfo=1/0/N:1,7,2,3,26,25,27,24,4,18,21,9,6...",Q92887_WT,...,NaN,=,4.940,4.940,0.000,0.000,1.0,4.940,0.000,Non-substrate
46983,AJUXEUPZWUBXHN_on_P08183_WT,High,ChEMBL31,ChEMBL31.compound.236273,COC(=O)C1=C(O)C(=CC=Cc2ccccc2OC)CCC1,AJUXEUPZWUBXHN,AJUXEUPZWUBXHN-UHFFFAOYSA-N,InChI=1S/C18H20O4/c1-21-16-12-4-3-7-13(16)8-5-...,"""AuxInfo=1/0/N:19,1,14,15,10,21,13,11,9,20,22,...",P08183_WT,...,NaN,=,4.690; 4.690,4.690,0.000,0.000,2.0,4.690,0.000,Non-substrate


In [17]:
#Keep only the relevant column, rename them
val_efflux = val_efflux[['SMILES','connectivity','InChIKey','pchembl_value_Mean','status_efflux']]
val_efflux = val_efflux.rename(columns={'connectivity': 'inchi_connectivity','SMILES':'papyrus_SMILES'})

In [18]:
#Code classes
val_efflux['status_efflux'] = val_efflux['status_efflux'].replace({'Substrate':1, 'Non-substrate':0})


In [19]:
#Reset index
val_efflux = val_efflux.reset_index(drop=True)

In [21]:
#Save the influx molecules
val_efflux.to_csv('val_data/kadar_efflux_val.csv', index=True)